Columnwise inversion with Multiprocessing
------------------------------------------

* Same as columnwise_inversion.ipynb, but with Multiprocessing instead of Dask


In [6]:
import xarray as xr
import numpy as np
import scipy.sparse.linalg

gmres = scipy.sparse.linalg.gmres

ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc", chunks = 'auto')
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc", chunks = 'auto')
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]


In [15]:
# code below based on https://stackoverflow.com/questions/45526700/easy-parallelization-of-numpy-apply-along-axis
# for some reason it gives a list of Nones for individual_results with gmres_partial


import multiprocessing



def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """        
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, multiprocessing.cpu_count())]

    pool = multiprocessing.Pool()
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def unpacking_apply_along_axis(all_args):
    (func1d, axis, arr, args, kwargs) = all_args

In [17]:
M = bio_2.values
n = len(M)
eye = np.eye(n)

from functools import partial

def gmres1d(A, b, tol = 1e-8):
    return gmres(A, b, tol)[0]

gmres_partial = partial(gmres1d, A=M)


parallel_apply_along_axis(gmres_partial, 1, eye)


[None, None, None, None, None, None, None, None]